In [6]:
from astroquery.jplhorizons import Horizons
earth = Horizons(3)
earthelements = earth.elements()
earthephemerides = earth.ephemerides()
earthvectors = earth.vectors()

import os
os.chdir(r"c:/Users/micha/OneDrive/Desktop/")
from vectorclass import Vector
pos_vector = Vector(*[0.9636433365004354	, 0.2637489304023757, 	-1.875334083261667e-5])
import math
math.sqrt((0.9636433365004354**2) +((0.2637489304023757)** 2) + ((-1.875334083261667e-5) ** 2))
pos_vector.get_magnitude() 
speedvector = Vector(earthvectors["vx"], earthvectors["vy"], earthvectors["vz"])
speedvector.get_magnitude()


math.sqrt((earthvectors["vx"])**2 +(earthvectors["vy"]** 2) + (earthvectors["vz"] ** 2))  *  1.496e+8 / 86400 # convert form au/d to km/s


29.835294169376336

### Comparing JLP to spiceypy


In [7]:
import spiceypy
import datetime
from astropy.time import Time
from astroquery.jplhorizons import Horizons
import astropy.units as u
import os

kerneldir = r"c:/Users/micha/OneDrive/Desktop/kernels"
os.chdir(r"c:/Users/micha/OneDrive/Desktop/kernels")
for i in os.listdir():
    filename = (os.path.join(kerneldir,i ))
    spiceypy.furnsh(filename)


comparison_target = 399

'''
INIT_TIME_UTC = datetime.datetime(year=2000, month=1, day=1, \
                                  hour=0, minute=0, second=0)
DELTA_DAYS = 10000
END_TIME_UTC = INIT_TIME_UTC + datetime.timedelta(days=DELTA_DAYS)
INIT_TIME_UTC_STR = INIT_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')
END_TIME_UTC_STR = END_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')
INIT_TIME_ET = spiceypy.utc2et(INIT_TIME_UTC_STR)
END_TIME_ET = spiceypy.utc2et(END_TIME_UTC_STR)
TIME_INTERVAL_ET = np.linspace(INIT_TIME_ET, END_TIME_ET, DELTA_DAYS)

'''

_, RADII_SUN = spiceypy.bodvcd(bodyid=10, item='RADII', maxn=3)
RADIUS_SUN = RADII_SUN[0]

au = 1.496e+8  # km

DATE_TODAY = datetime.datetime.today() # or datetime.datetime.now()
midnight = DATE_TODAY.strftime('%Y-%m-%dT00:00:00')
ET_TODAY_MIDNIGHT = spiceypy.utc2et(midnight)
onehourlater = datetime.datetime.fromisoformat(midnight).timestamp() + 3600 
endtime = (datetime.datetime.fromtimestamp(onehourlater))
epochs = {'start': str(midnight), 'stop': str(endtime), 'step': "1h"}

# SPICEY DATA
EARTH_STATE_WRT_SUN, EARTH_SUN_LIGHT_TIME = spiceypy.spkgeo(targ=comparison_target, et=ET_TODAY_MIDNIGHT, ref='ECLIPJ2000', obs=10)
# JET PROPULTION LAB DATA 
earth_vector = (Horizons(id=comparison_target, location="@10", epochs=epochs).vectors())


EARTH_SUN_DISTANCE = Vector(*EARTH_STATE_WRT_SUN[:3]).get_magnitude()
EARTH_SUN_angular_velocity = Vector(*EARTH_STATE_WRT_SUN[3:]).get_magnitude()
EARTH_SUN_angular_velocity_2 = math.sqrt((  (earth_vector[0]["vx"] *(au/86400) )**2.0) + ((earth_vector[0]["vy"] * (au/86400))**2.0) + ((earth_vector[0]["vz"] * (au/86400))**2.0))


#EARTH_SUN_DISTANCE = math.sqrt(EARTH_STATE_WRT_SUN[0]**2.0 + EARTH_STATE_WRT_SUN[1]**2.0 + EARTH_STATE_WRT_SUN[2]**2.0)
#EARTH_SUN_DISTANCE_AU = spiceypy.convrt(EARTH_SUN_DISTANCE, 'km', 'AU')
#EARTH_SUN_angular_velocity = math.sqrt(EARTH_STATE_WRT_SUN[3]**2.0 + EARTH_STATE_WRT_SUN[4]**2.0 + EARTH_STATE_WRT_SUN[5]**2.0)

print("comparing the xyz postion and range between spiceypy and JPL")
print((earth_vector[0]["x"] * au ) - (EARTH_STATE_WRT_SUN[0]) ) 
print((earth_vector[0]["y"] * au) -(EARTH_STATE_WRT_SUN[1]))
print((earth_vector[0]["z"] * au )- (EARTH_STATE_WRT_SUN[2]))
print((EARTH_SUN_DISTANCE) - (earth_vector[0]["range"]) * au)

_, GM_SUN = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
V_ORB_FUNC = lambda gm, r: math.sqrt(gm/r)
EARTH_ORB_SPEED_WRT_SUN_THEORY = V_ORB_FUNC(GM_SUN[0], EARTH_SUN_DISTANCE)


print("comparing the xyz velocities and angular velocity between spiceypy and JPL and theoretical  law")

print((earth_vector[0]["vx"] * (au/86400) ) - (EARTH_STATE_WRT_SUN[3]) ) # km per second
print((earth_vector[0]["vy"] * (au/86400) ) - (EARTH_STATE_WRT_SUN[4]))
print((earth_vector[0]["vz"] * (au/86400) )-  (EARTH_STATE_WRT_SUN[5]))
print(  EARTH_SUN_angular_velocity,  EARTH_SUN_angular_velocity_2, EARTH_ORB_SPEED_WRT_SUN_THEORY )



comparing the xyz postion and range between spiceypy and JPL
2677.0044373571873
-1324.8098164200783
0.1600827863981067
-2160.159504264593
comparing the xyz velocities and angular velocity between spiceypy and JPL and theoretical  law
0.0002576322633345285
0.0005244862688122964
1.6356543254807815e-08
29.817490416628612 29.817907501800644 29.80761152654426


### orbital velocity vector problem

In [8]:
postion_vector_jpl = Vector(*[a[0]["x"] , a[0]["y"]  , a[0]["z"] ])
orbital_velocity = V_ORB_FUNC(GM_SUN[0], (postion_vector_jpl  * au).get_magnitude())  # # r is jpl r in au then converted to km by multiplying by au

print("wrong the velocity vector is not the position vector",postion_vector_jpl.normalize() * orbital_velocity )
print("correct",Vector(*[a[0]["vx"] , a[0]["vy"]  , a[0]["vz"] ]) * au/86400)

wrong the velocity vector is not the position vector 28.451356661918094i + 8.888259641061111j + -0.0005926257765418207k
correct -9.361080054035714i + 28.310382931877463j + -0.002501666445675597k


In [9]:
## Orbital Period Equation

massofsun = 1.989 * 10**30
G = 6.673 * 10**-11 # m3 kg−1 s−2
radius_of_earth =6.37 * 10**6 # meters

r  = Vector(*[a[0]["x"] , a[0]["y"]  , a[0]["z"] ]).get_magnitude() * au * 1000 # in meters 
T = math.sqrt (   (4 * math.pi**2 * r**3) / (G*massofsun)   ) 
print(T / 86400)   # seconds to days conversion factor
T = math.sqrt (   (4 * math.pi**2 * r**3) / (GM_SUN[0] *10**9)   ) # GM_sun is in km while r is in m
print(T / 86400)  

print(Horizons(399).elements()["P"])

# a geostationary satellite completes one orbit per day above the equator, or 360 degrees per 24 hours, and has angular velocity ω = (360°)/(24 h) = 15°/h, or (2π rad)/(24 h) ≈ 0.26 rad/h
# If angle is measured in radians, the linear velocity v is the radius r times the angular velocity, v=rω
# for the satalite at r = 42,000 km from the earth's center, the satellite's speed through space is thus v = 42,000 km × 0.26 rad/h ≈ 11,000 km/h.


364.4043154101694
364.4228903763746
        P        
        d        
-----------------
364.7933956875075


In [10]:
acceleration_step = 1 # second

class SolarSystem:
    def __init__(self):
        self.bodies = []

    def add_body(self, body):
        self.bodies.append(body)
    def update_all(self):
        self.bodies.sort(key=lambda item: item.position[0])
        for body in self.bodies:
            body.move()
    def calculate_all_body_interactions(self):
        bodies_copy = self.bodies.copy()
        for idx, first in enumerate(bodies_copy):
            for second in bodies_copy[idx + 1:]:
                first.accelerate_due_to_gravity(second)

class SolarSystemBody:
    def __init__(self,solar_system,mass,position=(0, 0, 0),velocity=(0, 0, 0),):
        self.solar_system = solar_system
        self.mass = mass
        self.position = position
        self.velocity = Vector(*velocity)
        self.solar_system.add_body(self)
    def move(self):
        self.position = (
            self.position[0] + self.velocity[0],
            self.position[1] + self.velocity[1],
            self.position[2] + self.velocity[2],
        )

    def accelerate_due_to_gravity(self, other):
        global gravitational_constant 
        distance = Vector(*other.position) - Vector(*self.position)
        distance_mag = distance.get_magnitude()
        force_mag = gravitational_constant * (self.mass * other.mass / (distance_mag ** 2))
        force = distance.normalize() * force_mag 
        reverse = 1
        for body in self, other:
            acceleration = force / (body.mass/acceleration_step)          # accelaration in a day  # 86400
            body.velocity += acceleration * reverse
            reverse = -1

class Sun(SolarSystemBody):
    def __init__(self,solar_system,mass=10_000,position=(0, 0, 0),velocity=(0, 0, 0),):
        super(Sun, self).__init__(solar_system, mass, position, velocity)

class Planet(SolarSystemBody):
    def __init__(self,solar_system,mass=10_000,position=(0, 0, 0),velocity=(0, 0, 0),):
        super(Planet, self).__init__(solar_system, mass, position, velocity)



In [11]:
## extremely inaccurate analytical calculation of the solar system  


acceleration_step = 1 # second
number_of_steps = 36000 # 

DATE_TODAY = datetime.datetime.today() # or datetime.datetime.now()
midnight = DATE_TODAY.strftime('%Y-%m-%dT00:00:00')
ET_TODAY_MIDNIGHT = spiceypy.utc2et(midnight)
later = datetime.datetime.fromisoformat(midnight).timestamp() + number_of_steps+1
endtime = (datetime.datetime.fromtimestamp(later))
epo = {'start': str(midnight), 'stop': str(endtime), 'step': "1h"}

index_of_result = int(number_of_steps*acceleration_step / 3600) # 60 is seconds in 1m

au = 1.496e11
gravitational_constant = 6.6743 * 10**-11  #* 2.98692e-34
sunmass = 1.989 * 10**30 
moonmass = 7.34767309 * 10**22 #kilograms

namelist = [ "MERCURY" ,"VENUS" ,"EARTH", "MOON", "MARS","JUPITER","SATURN","URANUS", "NEPTUNE","PLUTO" ]
masslist = [0.330,4.87,5.97 ,0.073,0.642,1898,568,86.8,102,0.0130] # 10**24
#idlist = [199,299,399,499,599,699,799,899,999]  # barycenters location 
idlist = list(range(1,10))

central_location = "@10"


solar_system = SolarSystem()
Sun(solar_system, mass=sunmass , position=(0, 0, 0), velocity=(0, 0, 0))


def add_planet(i):
    end_vector_dict, planet_instence_dict = {}, {}
    planet_vector = (Horizons(idlist[i],location=central_location, epochs=epo)).vectors()
    postion_vector  =Vector(*[planet_vector["x"][0],planet_vector["y"][0],planet_vector["z"][0]]) * au
    velocity_vector = Vector(*[planet_vector["vx"][0],planet_vector["vy"][0],planet_vector["vz"][0]]) * au/86400
    planet_name = namelist[i]
    end_vector = Vector(planet_vector["x"][index_of_result] *au   ,(planet_vector["y"][index_of_result]) * au,(planet_vector["z"][index_of_result]) * au)
    planet = Planet(solar_system, masslist[i] * 10**24, position=postion_vector, velocity= velocity_vector)
    return end_vector, planet


results = []
for i in range(len(idlist)):
    end_vector, planet = add_planet(i)
    planetset = {}
    planetset[planet] = (end_vector)
    results.append(planetset)

for i in range(number_of_steps):
       solar_system.calculate_all_body_interactions()
       solar_system.update_all()

for  i in results:
    a, b = list(i.keys()), list(i.values())
    print(Vector(*a[0].position) - b[0])

2673.757827758789i + -10673.611541748047j + -1113.184518814087k
2107.7853240966797i + 358.8281669616699j + -112.73599147796631k
-1091.666015625i + -339.50852966308594j + 3.8390394574962556k
-424.5888671875i + -322.8360900878906j + 5.324418783187866k
131.5081787109375i + -36.12354278564453j + -1.7771663665771484k
174.925048828125i + -47.4443359375j + -2.8561134338378906k
154.551025390625i + 16.6982421875j + -2.0159912109375k
148.884765625i + 12.42926025390625j + -1.96234130859375k
150.2998046875i + 14.412109375j + -1.91461181640625k


### orbital elements:


In [377]:
import numpy as np

## orbital elements:
DATE_TODAY = datetime.datetime.today() # or datetime.datetime.now()
midnight = DATE_TODAY.strftime('%Y-%m-%dT00:00:00')
ET_TODAY_MIDNIGHT = spiceypy.utc2et(midnight)

state_vector, _ = spiceypy.spkgeo(targ=3, et=ET_TODAY_MIDNIGHT, ref='ECLIPJ2000',obs=10)
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

gm_sun = gm_sun_pre[0]
orbital_elements = spiceypy.oscltx(state=state_vector, et=ET_TODAY_MIDNIGHT, mu=gm_sun)

# Set and convert the semi-major axis and perihelion from km to AU
semi_major_au = spiceypy.convrt(orbital_elements[9], \
                                      inunit='km', outunit='AU')
perihelion_au = spiceypy.convrt(orbital_elements[0], \
                                      inunit='km', outunit='AU')

# Set the eccentricity
ecc = orbital_elements[1]

# Set and convert miscellaneous angular values from radians to degrees:
# inc: Inclination
# lnode: Longitude of ascending node
# argp: Argument of perihelion
inc_deg = np.degrees(orbital_elements[2])
lnode_deg = np.degrees(orbital_elements[3])
argp_deg = np.degrees(orbital_elements[4])

# Set the orbit period. Convert from seconds to years
orb_time_years = orbital_elements[10] / (86400.0 * 365.0)


print('Earth\' Orbital Elements')
print(f'Semi-major axis in AU: {round(semi_major_au, 2)} (MPC: 2.77)')
print(f'Perihelion in AU: {round(perihelion_au, 2)} (MPC: 2.55)')
print(f'Eccentricity: {round(ecc, 2)} (MPC: 0.08)')
print(f'Inclination in degrees: {round(inc_deg, 1)} (MPC: 10.6)')
print(f'Long. of. asc. node in degrees: {round(lnode_deg, 1)} ' '(MPC: 80.3)')
print(f'Argument of perih. in degrees: {round(argp_deg, 1)} ' '(MPC: 73.7)')
print(f'Orbit period in years: {round(orb_time_years, 2)} ' '(MPC: 4.60)')

# Convert the orbital elements back to the state vector
state_re = spiceypy.conics([*orbital_elements[:7], gm_sun], ET_TODAY_MIDNIGHT)
print('State vector of Earth from the kernel:\n' f'{state_vector}')
print('State vector of Earth based on the determined orbital elements:\n' f'{state_re}')

#Horizons(399).elements()["P"]
#for i in orbital_elements:
#    print(i)


Earth' Orbital Elements
Semi-major axis in AU: 1.0 (MPC: 2.77)
Perihelion in AU: 0.98 (MPC: 2.55)
Eccentricity: 0.02 (MPC: 0.08)
Inclination in degrees: 0.0 (MPC: 10.6)
Long. of. asc. node in degrees: 174.0 (MPC: 80.3)
Argument of perih. in degrees: 289.1 (MPC: 73.7)
Orbit period in years: 1.0 (MPC: 4.60)
State vector of Earth from the kernel:
[ 1.43364129e+08  4.20909545e+07 -2.93621379e+03 -8.87557390e+00
  2.84697814e+01 -1.41139746e-03]
State vector of Earth based on the determined orbital elements:
[ 1.43364129e+08  4.20909545e+07 -2.93621379e+03 -8.87557390e+00
  2.84697814e+01 -1.41139746e-03]
